In [11]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [ ]:
pc = Pinecone(api_key="5ccb4471-1c97-4a18-9e43-029ec1c706b9")
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [12]:
import json
data = json.load(open("reviews.json"))
data["reviews"]

[{'professor': 'Dr. Sarah Thompson',
  'subject': 'Biology',
  'stars': 4,
  'review': 'Engaging lectures and fair grading. Highly recommended for intro biology courses.'},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Excellent instructor who explains complex concepts clearly. Always available for office hours.'},
 {'professor': 'Dr. Emily Rodriguez',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Knowledgeable but sometimes disorganized. Assignments can be unclear.'},
 {'professor': 'Prof. David Smith',
  'subject': 'History',
  'stars': 4,
  'review': 'Passionate about the subject. Makes historical events come alive in class.'},
 {'professor': 'Dr. Lisa Johnson',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Tough but fair. Her classes prepare you well for advanced chemistry courses.'},
 {'professor': 'Prof. Robert Brown',
  'subject': 'Mathematics',
  'stars': 2,
  'review': 'Moves too quickly through material. Not ve

In [13]:
processed_data = []
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

for review in data["reviews"]:
    response = client.embeddings.create(
        input=review["review"],  
        model="text-embedding-ada-002",
    )
   
  
    embedding = response.data[0].embedding
    
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [14]:
processed_data[0]

{'values': [-0.0066766473,
  0.00084611506,
  -0.0047850404,
  -0.019575166,
  -0.010901456,
  0.018639248,
  -0.018204244,
  -0.008561663,
  -0.029474793,
  -0.021552455,
  -0.014131028,
  0.012463514,
  -0.00045189302,
  0.0056682294,
  0.012166921,
  0.011494642,
  0.034879386,
  -0.012120784,
  -0.015778769,
  -0.008238706,
  -0.020524263,
  0.031399354,
  0.0026446246,
  -0.009761219,
  -0.0020646197,
  0.015291038,
  0.019588348,
  -0.025045665,
  -0.009029621,
  -0.0021667797,
  0.020458354,
  -0.008298025,
  -0.022251097,
  -0.026746135,
  -0.0012720561,
  0.0037898046,
  -0.0050684516,
  0.028262056,
  0.023674745,
  0.0074807447,
  0.015765587,
  0.004145717,
  0.015844679,
  -0.005111293,
  -0.021473363,
  -0.02588931,
  -0.001362682,
  -0.014117846,
  -0.015501949,
  0.0060076644,
  0.011283731,
  0.011020093,
  0.011758281,
  -0.0010174802,
  -0.020814266,
  -0.015093309,
  0.0056286836,
  0.010631226,
  0.0041556032,
  -0.02185564,
  -0.014974671,
  -0.02807751,
  -0.0384

In [15]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [16]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}